**Table of contents**<a id='toc0_'></a>    
- [Introductie](#toc1_1_)    
  - [05-03-2025](#toc1_2_)    
    - [Testing data maken](#toc1_2_1_)    
  - [06-03-2025](#toc1_3_)    
    - [Quality control](#toc1_3_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Introductie](#toc0_)
blablabla

## <a id='toc1_2_'></a>[05-03-2025](#toc0_)
### <a id='toc1_2_1_'></a>[Testing data maken](#toc0_)
Om commands te testen zonder dat deze lang duren heb ik test-subsets gemaakt van de data
```{shell}
cat /students/2024-2025/Thema07/metagenomics/zoutstress/sample_info/{sample} | \
    parallel "seqkit {sample} -p 0.1 {} | seqkit head -n 100000 > /students/2024-2025/Thema07/metagenomics/zoutstress/testing_data/{sample}
```

de testing_data folder bevat nu subsets van de data waar tools op kunnen worden getest

## <a id='toc1_3_'></a>[06-03-2025](#toc0_)
### <a id='toc1_3_1_'></a>[Quality control](#toc0_)

Bij illumina data gebruiken wij fastqc om de kwaliteit van de sequences te bepalen.
Omdat minion net anders werkt, zijn hier ook andere tools voor.
Wij hebben gekozen voor [NanoPlot:1.44.1](https://github.com/wdecoster/NanoPlot), deze werd aangeraden op meerdere bioinformatica forums.

Deze is gedownload voor het testen met het volgende command:
```{shell}
conda install -c bioconda nanoplot
```
Nu kan NanoPlot gebruikt worden om de kwaliteit van de sequenties te bepalen.

```{shell}
find /students/2024-2025/Thema07/metagenomics/zoutstress/testing_data/ -type d -wholename '*rep*' | \
    parallel 'type_seq=$(basename $(dirname {}));' \
        'sample_name=$(basename {});' \
        'NanoPlot --fastq {}/*fastq.gz -o /students/2024-2025/Thema07/metagenomics/zoutstress/testing/NanoPlot/$type_seq/$sample_name/'
```
Geeft de volgende error:
```
    raise self._exception
gzip.BadGzipFile: Not a gzipped file (b'@d')
```
Dit houd in dat ondanks dat de bestanden op .gz eindigen, ze niet gzipped zijn.
Dit kan opgelost worden door .gz achter de naam weg te halen.
rename -n 's/.fastq.gz/.fastq/'

```{shell}
find /students/2024-2025/Thema07/metagenomics/zoutstress/testing_data/ -type d -wholename '*rep*' | \
    parallel 'rename "s/.fastq.gz/.fastq/" {}/*.fastq.gz'
```

Nu zijn alle .gz extensies weg.

```{shell}
find /students/2024-2025/Thema07/metagenomics/zoutstress/testing_data/ -type d -wholename '*rep*' | \
    parallel 'type_seq=$(basename $(dirname {}));' \
        'sample_name=$(basename {});' \
        'NanoPlot -p "${type_seq}_${sample_name}_" --fastq {}/*.fastq' \ 
            '-o /students/2024-2025/Thema07/metagenomics/zoutstress/testing/NanoPlot/'
```

Dit genereerd voor alle samples (zoet/zout) en type sequencing (16s/wgs) een html bestand met informatie over de kwaliteit.
Omdat er aardig wat resulteren de bestanden zijn, ga ik deze samenvoegen tot 1 html bestand met Multiqc

```{shell}
multiqc /students/2024-2025/Thema07/metagenomics/zoutstress/testing/NanoPlot/* -o /students/2024-2025/Thema07/metagenomics/zoutstress/testing/Multiqc
```

Het resulterende html bestand bevat niet veel informatie.
Daarom gaan we voor het trimmen eerst kijken naar de losse html bestanden, gemaakt door NanoPlot

Als we kijken naar de kwaliteit van de 16s samples zien we het volgende:
Een gemiddelde read lengte van ~730.
Een gemiddelde read kwaliteit van ~12.7

Dit is een best lage kwaliteit, wat verwacht kan worden als sequencing met een minIon gedaan wordt.

Voor WGS zien we het volgende:
Gemiddelde read lengte van 1000-1100, behalve bij 1: 761
Gemiddelde kwaliteit van 10.4-11.1, behalve bij 1: 12.2

We moeten de reads nog trimmen om een hogere score te krijgen
Ik ga nanoplot en multiqc op de echte data uitvoeren wanneer deze gefixed zijn, er staat nog .gz achter.
